<a href="https://colab.research.google.com/github/aaditya3301/agents/blob/main/agenticai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random
from collections import deque
import matplotlib.pyplot as plt

# --- Helper Functions ---
DIRS = [(0, 1), (1, 0), (0, -1), (-1, 0)]

def in_bounds(pos, H, W):
    return 0 <= pos[0] < H and 0 <= pos[1] < W

def neighbors(pos, H, W, grid):
    for dr, dc in DIRS:
        nr, nc = pos[0] + dr, pos[1] + dc
        if in_bounds((nr, nc), H, W) and grid[nr, nc] == 0:
            yield (nr, nc)

def bfs(start, goals, H, W, grid):
    q = deque([start])
    parent = {start: None}
    while q:
        cur = q.popleft()
        if cur in goals:
            path = []
            p = cur
            while p:
                path.append(p)
                p = parent[p]
            return path[::-1]
        for nb in neighbors(cur, H, W, grid):
            if nb not in parent:
                parent[nb] = cur
                q.append(nb)
    return None

# --- Classes ---
class GridWorld:
    def __init__(self, H, W):
        self.H = H
        self.W = W
        self.grid = np.zeros((H, W), dtype=int)

    def add_random_walls(self, prob=0.18, seed=0):
        random.seed(seed)
        for r in range(self.H):
            for c in range(self.W):
                if random.random() < prob:
                    self.grid[r, c] = 1

class Agent:
    def __init__(self, id, start):
        self.id = id
        self.pos = start
        self.path = [start]
        self.history = [start]
        self.task = None

    def set_path(self, p):
        if p: self.path = p

    def step(self):
        if len(self.path) > 1:
            self.path = self.path[1:]
            self.pos = self.path[0]
            self.history.append(self.pos)

# --- Main Execution ---
H, W = 15, 15
world = GridWorld(H, W)
world.add_random_walls(0.18, seed=42)
starts = [(1, 1), (H-2, W-2)]
for s in starts: world.grid[s] = 0

# Place keys
keys = set()
while len(keys) < 8:
    p = (random.randrange(H), random.randrange(W))
    if world.grid[p] == 0 and p not in starts:
        keys.add(p)

agents = [Agent(0, starts[0]), Agent(1, starts[1])]
shared = set(keys)
t = 0
max_steps = 500

while shared and t < max_steps:
    # Claim nearest key
    claims = {}
    for a in agents:
        if not shared: break
        nearest = min(shared, key=lambda x: abs(x[0]-a.pos[0]) + abs(x[1]-a.pos[1]))
        claims[a.id] = nearest

    # Resolve duplicates
    assigned = set()
    for a in agents:
        if a.id in claims:
            tgt = claims[a.id]
            if tgt in assigned:
                choices = [r for r in shared if r not in assigned]
                if choices:
                    tgt = min(choices, key=lambda x: abs(x[0]-a.pos[0]) + abs(x[1]-a.pos[1]))
            a.task = tgt
            assigned.add(tgt)

    # Move
    for a in agents:
        if a.task:
            p = bfs(a.pos, {a.task}, H, W, world.grid)
            if p: a.set_path(p)
        a.step()
        if a.pos in shared:
            shared.remove(a.pos)
    t += 1

# Visualization
canvas = np.ones((H, W, 3)) * 0.95
canvas[world.grid == 1] = np.array([0.1, 0.1, 0.1])
for k in keys: canvas[k] = np.array([0.8, 0.4, 0.0])
for a in agents:
    for (r, c) in a.history:
        canvas[r, c] = np.array([0.6, 0.6, 0.95])
for a in agents:
    r, c = a.pos
    canvas[r, c] = np.array([0.2 + 0.3 * (a.id % 3), 0.2 + 0.4 * ((a.id + 1) % 3), 0.3 + 0.3 * ((a.id + 2) % 3)])

print("Keys collected:", 8 - len(shared))
plt.figure(figsize=(6, 6))
plt.imshow(canvas)
plt.title('Dual Maze Navigators Final')
plt.axis('off')
plt.show()

In [ ]:
import numpy as np
import random
import heapq
import matplotlib.pyplot as plt

DIRS = [(0, 1), (1, 0), (0, -1), (-1, 0)]

def in_bounds(p, H, W): return 0 <= p[0] < H and 0 <= p[1] < W

def neighbors(p, H, W, grid):
    for dr, dc in DIRS:
        nr, nc = p[0] + dr, p[1] + dc
        if in_bounds((nr, nc), H, W) and grid[nr, nc] == 0:
            yield (nr, nc)

def manhattan(a, b): return abs(a[0] - b[0]) + abs(a[1] - b[1])

def astar(start, goal, H, W, grid):
    open_set = []
    heapq.heappush(open_set, (manhattan(start, goal), 0, start, None))
    came = {}
    g = {start: 0}

    while open_set:
        f, dist, cur, parent = heapq.heappop(open_set)
        if cur in came: continue
        came[cur] = parent

        if cur == goal:
            path = []
            p = cur
            while p:
                path.append(p)
                p = came[p] if p in came else None
            return path[::-1]

        for nb in neighbors(cur, H, W, grid):
            nd = dist + 1
            if nb not in g or nd < g[nb]:
                g[nb] = nd
                heapq.heappush(open_set, (nd + manhattan(nb, goal), nd, nb, cur))
    return None

class GridWorld:
    def __init__(self, H, W):
        self.H, self.W = H, W
        self.grid = np.zeros((H, W), dtype=int)

    def add_walls(self, prob=0.08, seed=0):
        random.seed(seed)
        for r in range(self.H):
            for c in range(self.W):
                if random.random() < prob: self.grid[r, c] = 1

class Agent:
    def __init__(self, id, start):
        self.id = id
        self.pos = start
        self.history = [start]
        self.path = [start]

    def step(self):
        if len(self.path) > 1:
            self.path = self.path[1:]
            self.pos = self.path[0]
            self.history.append(self.pos)

H, W = 12, 12
world = GridWorld(H, W)
world.add_walls(prob=0.08, seed=7)

dirty = [(r, c) for r in range(H) for c in range(W) if world.grid[r, c] == 0 and random.random() < 0.25]
dirty_set = set(dirty)

a1 = Agent(0, (0, 0))
a2 = Agent(1, (H-1, W-1))
agents = [a1, a2]

# Divide regions
region1 = {(r, c) for r in range(H) for c in range(W) if c < W // 2}
region2 = {(r, c) for r in range(H) for c in range(W) if c >= W // 2}

t = 0
while dirty_set and t < 500:
    t += 1
    for a, region in [(a1, region1), (a2, region2)]:
        region_targets = [d for d in dirty_set if d in region]
        if not region_targets:
            region_targets = list(dirty_set) # Help other region if done
        if not region_targets: continue

        target = min(region_targets, key=lambda x: manhattan(a.pos, x))
        p = astar(a.pos, target, H, W, world.grid)
        if p: a.path = p

        a.step()
        if a.pos in dirty_set: dirty_set.remove(a.pos)

# Visualize
canvas = np.ones((H, W, 3)) * 0.95
canvas[world.grid == 1] = np.array([0.1, 0.1, 0.1])
for d in dirty: canvas[d] = np.array([0.6, 0.3, 0.1])
for a in agents:
    for p in a.history: canvas[p] = np.array([0.8, 0.9, 0.9])
    r, c = a.pos
    canvas[r, c] = np.array([0.2 + 0.3 * (a.id % 3), 0.2 + 0.4 * ((a.id + 1) % 3), 0.3 + 0.3 * ((a.id + 2) % 3)])

plt.figure(figsize=(6, 6))
plt.imshow(canvas)
plt.title('Cleaning Crew Final')
plt.axis('off')
plt.show()

In [ ]:
import numpy as np
import random
import heapq
import matplotlib.pyplot as plt

DIRS = [(0, 1), (1, 0), (0, -1), (-1, 0)]

def in_bounds(p, H, W): return 0 <= p[0] < H and 0 <= p[1] < W

def neighbors(p, H, W, grid):
    for dr, dc in DIRS:
        nr, nc = p[0] + dr, p[1] + dc
        if in_bounds((nr, nc), H, W) and grid[nr, nc] == 0: yield (nr, nc)

def manhattan(a, b): return abs(a[0] - b[0]) + abs(a[1] - b[1])

def astar(start, goal, H, W, grid):
    open_set = []
    heapq.heappush(open_set, (manhattan(start, goal), 0, start, None))
    came = {}
    g = {start: 0}
    while open_set:
        f, dist, cur, parent = heapq.heappop(open_set)
        if cur in came: continue
        came[cur] = parent
        if cur == goal:
            path = []
            p = cur
            while p:
                path.append(p)
                p = came[p] if p in came else None
            return path[::-1]
        for nb in neighbors(cur, H, W, grid):
            nd = dist + 1
            if nb not in g or nd < g[nb]:
                g[nb] = nd
                heapq.heappush(open_set, (nd + manhattan(nb, goal), nd, nb, cur))
    return None

class GridWorld:
    def __init__(self, H, W):
        self.H, self.W = H, W
        self.grid = np.zeros((H, W), dtype=int)
    def add_walls(self, prob=0.12, seed=0):
        random.seed(seed)
        for r in range(self.H):
            for c in range(self.W):
                if random.random() < prob: self.grid[r, c] = 1

class Agent:
    def __init__(self, id, start):
        self.id = id
        self.pos = start
        self.history = [start]
    def set_history(self, h):
        self.history = h
        self.pos = h[-1]

H, W = 12, 12
world = GridWorld(H, W)
world.add_walls(prob=0.12, seed=5)

start1, start2 = (1, 1), (1, W-2)
goal1, goal2 = (H-2, 1), (H-2, W-2)

# Plan independent paths
p1 = astar(start1, goal1, H, W, world.grid) or [start1]
p2 = astar(start2, goal2, H, W, world.grid) or [start2]
hist = {0: list(p1), 1: list(p2)}

# Collision Resolution
t = 0
while t < 200:
    pos0 = hist[0][t] if t < len(hist[0]) else hist[0][-1]
    pos1 = hist[1][t] if t < len(hist[1]) else hist[1][-1]

    if pos0 == pos1:
        # Agent 1 waits one step
        hist[1].insert(t, hist[1][t-1] if t > 0 else hist[1][0])
    t += 1

agents = [Agent(0, start1), Agent(1, start2)]
for i, a in enumerate(agents): a.set_history(hist[i])

# Visualize
canvas = np.ones((H, W, 3)) * 0.95
canvas[world.grid == 1] = np.array([0.1, 0.1, 0.1])
for a in agents:
    for p in a.history: canvas[p] = np.array([0.8, 0.8, 0.6])
    r, c = a.pos
    canvas[r, c] = np.array([0.2 + 0.3 * (a.id % 3), 0.2 + 0.4 * ((a.id + 1) % 3), 0.3 + 0.3 * ((a.id + 2) % 3)])

plt.figure(figsize=(6, 6))
plt.imshow(canvas)
plt.title('Cooperative Path Planners Final')
plt.axis('off')
plt.show()

In [ ]:
import numpy as np
import random
import heapq
import matplotlib.pyplot as plt

DIRS = [(0, 1), (1, 0), (0, -1), (-1, 0)]

def in_bounds(p, H, W): return 0 <= p[0] < H and 0 <= p[1] < W

def neighbors(p, H, W, grid):
    for dr, dc in DIRS:
        nr, nc = p[0] + dr, p[1] + dc
        if in_bounds((nr, nc), H, W) and grid[nr, nc] == 0: yield (nr, nc)

def manhattan(a, b): return abs(a[0] - b[0]) + abs(a[1] - b[1])

def astar(start, goal, H, W, grid):
    open_set = []
    heapq.heappush(open_set, (manhattan(start, goal), 0, start, None))
    came = {}
    g = {start: 0}
    while open_set:
        f, dist, cur, parent = heapq.heappop(open_set)
        if cur in came: continue
        came[cur] = parent
        if cur == goal:
            path = []
            p = cur
            while p:
                path.append(p)
                p = came[p] if p in came else None
            return path[::-1]
        for nb in neighbors(cur, H, W, grid):
            nd = dist + 1
            if nb not in g or nd < g[nb]:
                g[nb] = nd
                heapq.heappush(open_set, (nd + manhattan(nb, goal), nd, nb, cur))
    return None

class GridWorld:
    def __init__(self, H, W):
        self.H, self.W = H, W
        self.grid = np.zeros((H, W), dtype=int)
    def add_walls(self, prob=0.05, seed=0):
        random.seed(seed)
        for r in range(self.H):
            for c in range(self.W):
                if random.random() < prob: self.grid[r, c] = 1

class Agent:
    def __init__(self, id, start):
        self.id = id
        self.pos = start
        self.path = [start]
        self.history = [start]
        self.task = None
    def step(self):
        if len(self.path) > 1:
            self.path = self.path[1:]
            self.pos = self.path[0]
            self.history.append(self.pos)

H, W = 14, 14
world = GridWorld(H, W)
world.add_walls(prob=0.05, seed=11)
starts = [(0, 0), (H-1, 0), (0, W-1)]
agents = [Agent(i, s) for i, s in enumerate(starts)]
items = [(random.randint(1, H-2), random.randint(1, W-2)) for _ in range(8)]
remaining = set(items)

t = 0
while remaining and t < 400:
    remaining_list = list(remaining)
    for a in agents:
        if a.task is None and remaining_list:
            a.task = min(remaining_list, key=lambda x: manhattan(a.pos, x))

    for a in agents:
        if a.task:
            p = astar(a.pos, a.task, H, W, world.grid)
            if p: a.path = p
        a.step()
        if a.pos == a.task:
            remaining.discard(a.task)
            a.task = None
    t += 1

canvas = np.ones((H, W, 3)) * 0.95
canvas[world.grid == 1] = np.array([0.1, 0.1, 0.1])
for it in items: canvas[it] = np.array([0.8, 0.4, 0.0])
for a in agents:
    for p in a.history: canvas[p] = np.array([0.7, 0.9, 0.9])
    r, c = a.pos
    canvas[r, c] = np.array([0.2 + 0.3 * (a.id % 3), 0.2 + 0.4 * ((a.id + 1) % 3), 0.3 + 0.3 * ((a.id + 2) % 3)])

plt.figure(figsize=(6, 6))
plt.imshow(canvas)
plt.title('Warehouse Pickup Final')
plt.axis('off')
plt.show()

In [ ]:
import numpy as np
import random
from collections import deque
import matplotlib.pyplot as plt

# --- Constants & Helpers ---
DIRS = [(0, 1), (1, 0), (0, -1), (-1, 0)]

def in_bounds(p, H, W):
    return 0 <= p[0] < H and 0 <= p[1] < W

def neighbors(p, H, W, grid):
    for dr, dc in DIRS:
        nr, nc = p[0] + dr, p[1] + dc
        if in_bounds((nr, nc), H, W) and grid[nr, nc] == 0:
            yield (nr, nc)

def bfs(start, goals, H, W, grid):
    q = deque([start])
    parent = {start: None}
    while q:
        cur = q.popleft()
        if cur in goals:
            path = []
            p = cur
            while p:
                path.append(p)
                p = parent[p]
            return path[::-1]
        for nb in neighbors(cur, H, W, grid):
            if nb not in parent:
                parent[nb] = cur
                q.append(nb)
    return None

# --- Classes ---
class GridWorld:
    def __init__(self, H, W):
        self.H, self.W = H, W
        self.grid = np.zeros((H, W), dtype=int)

    def add_walls(self, prob=0.10, seed=0):
        random.seed(seed)
        for r in range(self.H):
            for c in range(self.W):
                if random.random() < prob:
                    self.grid[r, c] = 1

class Agent:
    def __init__(self, id, start):
        self.id = id
        self.pos = start
        self.path = [start]
        self.history = [start]

    def step(self):
        if len(self.path) > 1:
            self.path = self.path[1:]
            self.pos = self.path[0]
            self.history.append(self.pos)

# --- Main Execution ---
H, W = 15, 15
world = GridWorld(H, W)
world.add_walls(prob=0.10, seed=3)

starts = [(0, 0), (0, W-1), (H-1, 0)]
agents = [Agent(i, s) for i, s in enumerate(starts)]

victims = set()
while len(victims) < 6:
    p = (random.randrange(H), random.randrange(W))
    if world.grid[p] == 0 and p not in starts:
        victims.add(p)

remaining = set(victims)
t = 0

while remaining and t < 400:
    t += 1
    for a in agents:
        # Re-plan if idle or path finished
        if not a.path or len(a.path) <= 1:
            path = bfs(a.pos, remaining, H, W, world.grid)
            if path: a.path = path

        a.step()

        if a.pos in remaining:
            remaining.remove(a.pos)

print(f"Rescued {len(victims)-len(remaining)}/{len(victims)} victims.")

# Visualization
canvas = np.ones((H, W, 3)) * 0.95
canvas[world.grid == 1] = np.array([0.1, 0.1, 0.1])
for v in victims: canvas[v] = np.array([0.9, 0.4, 0.4])
for a in agents:
    for p in a.history: canvas[p] = np.array([0.7, 0.9, 0.9])
    r, c = a.pos
    canvas[r, c] = np.array([0.2 + 0.3 * (a.id % 3), 0.2 + 0.4 * ((a.id + 1) % 3), 0.3 + 0.3 * ((a.id + 2) % 3)])

plt.figure(figsize=(6, 6))
plt.imshow(canvas)
plt.title('Rescue Bot Squad Final')
plt.axis('off')
plt.show()

In [ ]:
import numpy as np
import random
import heapq
import matplotlib.pyplot as plt

# --- Constants & Helpers ---
DIRS = [(0, 1), (1, 0), (0, -1), (-1, 0)]

def in_bounds(p, H, W): return 0 <= p[0] < H and 0 <= p[1] < W

def neighbors(p, H, W, grid):
    for dr, dc in DIRS:
        nr, nc = p[0] + dr, p[1] + dc
        if in_bounds((nr, nc), H, W) and grid[nr, nc] == 0: yield (nr, nc)

def manhattan(a, b): return abs(a[0] - b[0]) + abs(a[1] - b[1])

def astar(start, goal, H, W, grid):
    open_set = []
    heapq.heappush(open_set, (manhattan(start, goal), 0, start, None))
    came = {}
    g = {start: 0}
    while open_set:
        f, dist, cur, parent = heapq.heappop(open_set)
        if cur in came: continue
        came[cur] = parent
        if cur == goal:
            path = []
            p = cur
            while p:
                path.append(p)
                p = came[p] if p in came else None
            return path[::-1]
        for nb in neighbors(cur, H, W, grid):
            nd = dist + 1
            if nb not in g or nd < g[nb]:
                g[nb] = nd
                heapq.heappush(open_set, (nd + manhattan(nb, goal), nd, nb, cur))
    return None

# --- Classes ---
class GridWorld:
    def __init__(self, H, W):
        self.H, self.W = H, W
        self.grid = np.zeros((H, W), dtype=int)
    def add_walls(self, prob=0.04, seed=0):
        random.seed(seed)
        for r in range(self.H):
            for c in range(self.W):
                if random.random() < prob: self.grid[r, c] = 1

class Agent:
    def __init__(self, id, start):
        self.id = id
        self.pos = start
        self.path = [start]
        self.history = [start]
        self.task = None
    def step(self):
        if len(self.path) > 1:
            self.path = self.path[1:]
            self.pos = self.path[0]
            self.history.append(self.pos)

# --- Main Execution ---
H, W = 16, 16
world = GridWorld(H, W)
world.add_walls(prob=0.04, seed=99)

drones = [Agent(0, (1, 1)), Agent(1, (H-2, W-2))]
packages = [(random.randint(1, H-2), random.randint(1, W-2)) for _ in range(6)]
remaining = set(packages)

t = 0
while remaining and t < 400:
    t += 1
    # Greedy assignment
    for d in drones:
        if d.task is None and remaining:
            d.task = min(remaining, key=lambda x: manhattan(d.pos, x))

    for d in drones:
        if d.task:
            p = astar(d.pos, d.task, H, W, world.grid)
            if p: d.path = p
        d.step()
        if d.task and d.pos == d.task:
            remaining.discard(d.task)
            d.task = None

print(f"Delivered {len(packages)-len(remaining)}/{len(packages)} packages.")

# Visualization
canvas = np.ones((H, W, 3)) * 0.95
canvas[world.grid == 1] = np.array([0.1, 0.1, 0.1])
for pk in packages: canvas[pk] = np.array([0.8, 0.6, 0.2])
for d in drones:
    for p in d.history: canvas[p] = np.array([0.7, 0.9, 0.9])
    r, c = d.pos
    canvas[r, c] = np.array([0.2 + 0.3 * (d.id % 3), 0.2 + 0.4 * ((d.id + 1) % 3), 0.3 + 0.3 * ((d.id + 2) % 3)])

plt.figure(figsize=(6, 6))
plt.imshow(canvas)
plt.title('Dual Drone Delivery Final')
plt.axis('off')
plt.show()

In [ ]:
import numpy as np
import random
from collections import deque
import matplotlib.pyplot as plt

DIRS = [(0, 1), (1, 0), (0, -1), (-1, 0)]

def in_bounds(p, H, W): return 0 <= p[0] < H and 0 <= p[1] < W

def neighbors(p, H, W, grid):
    for dr, dc in DIRS:
        nr, nc = p[0] + dr, p[1] + dc
        if in_bounds((nr, nc), H, W) and grid[nr, nc] == 0: yield (nr, nc)

def bfs(start, goals, H, W, grid):
    q = deque([start])
    parent = {start: None}
    while q:
        cur = q.popleft()
        if cur in goals:
            path = []
            p = cur
            while p:
                path.append(p)
                p = parent[p]
            return path[::-1]
        for nb in neighbors(cur, H, W, grid):
            if nb not in parent:
                parent[nb] = cur
                q.append(nb)
    return None

class GridWorld:
    def __init__(self, H, W):
        self.H, self.W = H, W
        self.grid = np.zeros((H, W), dtype=int)
    def add_walls(self, prob=0.05, seed=0):
        random.seed(seed)
        for r in range(self.H):
            for c in range(self.W):
                if random.random() < prob: self.grid[r, c] = 1

class Agent:
    def __init__(self, id, start):
        self.id = id
        self.pos = start
        self.path = [start]
        self.history = [start]
    def step(self):
        if len(self.path) > 1:
            self.path = self.path[1:]
            self.pos = self.path[0]
            self.history.append(self.pos)

# --- Main Execution ---
H, W = 12, 12
world = GridWorld(H, W)
world.add_walls(prob=0.05, seed=8)

painters = [Agent(0, (0, 0)), Agent(1, (H-1, W-1))]
to_paint = {(r, c) for r in range(H) for c in range(W) if world.grid[r, c] == 0 and random.random() < 0.35}

# Checkerboard partition to avoid conflict
region1 = {p for p in to_paint if (p[0] + p[1]) % 2 == 0}
region2 = to_paint - region1
rem1, rem2 = set(region1), set(region2)

t = 0
while (rem1 or rem2) and t < 500:
    t += 1
    for idx, (a, rem) in enumerate([(painters[0], rem1), (painters[1], rem2)]):
        if not rem: continue

        # Find nearest unpainted cell in own region
        target = min(rem, key=lambda x: abs(x[0] - a.pos[0]) + abs(x[1] - a.pos[1]))
        p = bfs(a.pos, {target}, H, W, world.grid)
        if p: a.path = p

        a.step()
        if a.pos in rem:
            rem.remove(a.pos)

print(f"Painted approx {len(to_paint) - len(rem1) - len(rem2)} cells.")

# Visualization
canvas = np.ones((H, W, 3)) * 0.95
canvas[world.grid == 1] = np.array([0.1, 0.1, 0.1])
for p in to_paint: canvas[p] = np.array([0.95, 0.9, 0.7]) # base paint color
for a in painters:
    for h in a.history: canvas[h] = np.array([0.8, 0.9, 0.9]) # path
    r, c = a.pos
    canvas[r, c] = np.array([0.2 + 0.3 * (a.id % 3), 0.2 + 0.4 * ((a.id + 1) % 3), 0.3 + 0.3 * ((a.id + 2) % 3)])

plt.figure(figsize=(6, 6))
plt.imshow(canvas)
plt.title('Grid Painting Final')
plt.axis('off')
plt.show()

In [ ]:
import numpy as np
import random
import heapq
import matplotlib.pyplot as plt

DIRS = [(0, 1), (1, 0), (0, -1), (-1, 0)]

def in_bounds(p, H, W): return 0 <= p[0] < H and 0 <= p[1] < W

def neighbors(p, H, W, grid):
    for dr, dc in DIRS:
        nr, nc = p[0] + dr, p[1] + dc
        if in_bounds((nr, nc), H, W) and grid[nr, nc] == 0: yield (nr, nc)

def manhattan(a, b): return abs(a[0] - b[0]) + abs(a[1] - b[1])

def astar(start, goal, H, W, grid):
    open_set = []
    heapq.heappush(open_set, (manhattan(start, goal), 0, start, None))
    came = {}
    g = {start: 0}
    while open_set:
        f, dist, cur, parent = heapq.heappop(open_set)
        if cur in came: continue
        came[cur] = parent
        if cur == goal:
            path = []
            p = cur
            while p:
                path.append(p)
                p = came[p] if p in came else None
            return path[::-1]
        for nb in neighbors(cur, H, W, grid):
            nd = dist + 1
            if nb not in g or nd < g[nb]:
                g[nb] = nd
                heapq.heappush(open_set, (nd + manhattan(nb, goal), nd, nb, cur))
    return None

class GridWorld:
    def __init__(self, H, W):
        self.H, self.W = H, W
        self.grid = np.zeros((H, W), dtype=int)
    def add_walls(self, prob=0.07, seed=0):
        random.seed(seed)
        for r in range(self.H):
            for c in range(self.W):
                if random.random() < prob: self.grid[r, c] = 1

class Agent:
    def __init__(self, id, start):
        self.id = id
        self.pos = start
        self.path = [start]
        self.history = [start]
        self.task = None
    def step(self):
        if len(self.path) > 1:
            self.path = self.path[1:]
            self.pos = self.path[0]
            self.history.append(self.pos)

# --- Main Execution ---
H, W = 14, 14
world = GridWorld(H, W)
world.add_walls(prob=0.07, seed=13)

agents = [Agent(0, (0, 0)), Agent(1, (H-1, W-1)), Agent(2, (H-1, 0))]
resources = {(random.randint(1, H-2), random.randint(1, W-2)) for _ in range(10)}
task_queue = list(resources)

t = 0
while task_queue and t < 600:
    t += 1
    for a in agents:
        if a.task is None and task_queue:
            a.task = task_queue.pop(0)

        if a.task:
            p = astar(a.pos, a.task, H, W, world.grid)
            if p: a.path = p

        a.step()
        if a.pos == a.task:
            a.task = None

print(f"Collected resources: approx {len(resources)} (distributed among agents).")

# Visualization
canvas = np.ones((H, W, 3)) * 0.95
canvas[world.grid == 1] = np.array([0.1, 0.1, 0.1])
for rsrc in resources: canvas[rsrc] = np.array([0.8, 0.5, 0.2])
for a in agents:
    for h in a.history: canvas[h] = np.array([0.8, 0.9, 0.9])
    r, c = a.pos
    canvas[r, c] = np.array([0.2 + 0.3 * (a.id % 3), 0.2 + 0.4 * ((a.id + 1) % 3), 0.3 + 0.3 * ((a.id + 2) % 3)])

plt.figure(figsize=(6, 6))
plt.imshow(canvas)
plt.title('Resource Collection Final')
plt.axis('off')
plt.show()

In [ ]:
import numpy as np
import random
from collections import deque
import matplotlib.pyplot as plt

DIRS = [(0, 1), (1, 0), (0, -1), (-1, 0)]

def in_bounds(p, H, W): return 0 <= p[0] < H and 0 <= p[1] < W

def neighbors(p, H, W, grid):
    for dr, dc in DIRS:
        nr, nc = p[0] + dr, p[1] + dc
        if in_bounds((nr, nc), H, W) and grid[nr, nc] == 0: yield (nr, nc)

def manhattan(a, b): return abs(a[0] - b[0]) + abs(a[1] - b[1])

def bfs(start, goals, H, W, grid):
    q = deque([start])
    parent = {start: None}
    while q:
        cur = q.popleft()
        if cur in goals:
            path = []
            p = cur
            while p:
                path.append(p)
                p = parent[p]
            return path[::-1]
        for nb in neighbors(cur, H, W, grid):
            if nb not in parent:
                parent[nb] = cur
                q.append(nb)
    return None

class GridWorld:
    def __init__(self, H, W):
        self.H, self.W = H, W
        self.grid = np.zeros((H, W), dtype=int)
    def add_walls(self, prob=0.03, seed=0):
        random.seed(seed)
        for r in range(self.H):
            for c in range(self.W):
                if random.random() < prob: self.grid[r, c] = 1

class Agent:
    def __init__(self, id, start):
        self.id = id
        self.pos = start
        self.path = [start]
        self.history = [start]
        self.task = None
    def step(self):
        if len(self.path) > 1:
            self.path = self.path[1:]
            self.pos = self.path[0]
            self.history.append(self.pos)

# --- Main Execution ---
H, W = 16, 16
world = GridWorld(H, W)
world.add_walls(prob=0.03, seed=21)

ffs = [Agent(0, (0, 0)), Agent(1, (0, W-1)), Agent(2, (H-1, W-1))]
fires = {(random.randint(3, H-4), random.randint(3, W-4)) for _ in range(6)}

t = 0
while fires and t < 300:
    # Greedy assignment
    for f in ffs:
        if f.task is None and fires:
            f.task = min(fires, key=lambda x: manhattan(f.pos, x))

    for f in ffs:
        if f.task:
            p = bfs(f.pos, {f.task}, H, W, world.grid)
            if p: f.path = p
        f.step()

        if f.pos == f.task and f.pos in fires:
            fires.remove(f.pos)
            f.task = None

    # Fire Spread
    newfires = set(fires)
    for fire in list(fires):
        for nb in neighbors(fire, H, W, world.grid):
            if random.random() < 0.08: # Spread probability
                newfires.add(nb)
    fires = newfires
    t += 1

print("Remaining fires approx:", len(fires))

# Visualization
canvas = np.ones((H, W, 3)) * 0.95
canvas[world.grid == 1] = np.array([0.1, 0.1, 0.1])
for fr in fires: canvas[fr] = np.array([0.9, 0.3, 0.2])
for f in ffs:
    for h in f.history: canvas[h] = np.array([0.8, 0.9, 0.9])
    r, c = f.pos
    canvas[r, c] = np.array([0.2 + 0.3 * (f.id % 3), 0.2 + 0.4 * ((f.id + 1) % 3), 0.3 + 0.3 * ((f.id + 2) % 3)])

plt.figure(figsize=(6, 6))
plt.imshow(canvas)
plt.title('Cooperative Firefighters Final')
plt.axis('off')
plt.show()

In [ ]:
import numpy as np
import random
from collections import deque
import matplotlib.pyplot as plt

DIRS = [(0, 1), (1, 0), (0, -1), (-1, 0)]

def in_bounds(p, H, W): return 0 <= p[0] < H and 0 <= p[1] < W

def neighbors(p, H, W, grid):
    for dr, dc in DIRS:
        nr, nc = p[0] + dr, p[1] + dc
        if in_bounds((nr, nc), H, W) and grid[nr, nc] == 0: yield (nr, nc)

def bfs(start, goals, H, W, grid):
    q = deque([start])
    parent = {start: None}
    while q:
        cur = q.popleft()
        if cur in goals:
            path = []
            p = cur
            while p:
                path.append(p)
                p = parent[p]
            return path[::-1]
        for nb in neighbors(cur, H, W, grid):
            if nb not in parent:
                parent[nb] = cur
                q.append(nb)
    return None

class GridWorld:
    def __init__(self, H, W):
        self.H, self.W = H, W
        self.grid = np.zeros((H, W), dtype=int)
    def add_walls(self, prob=0.06, seed=0):
        random.seed(seed)
        for r in range(self.H):
            for c in range(self.W):
                if random.random() < prob: self.grid[r, c] = 1

class Agent:
    def __init__(self, id, start):
        self.id = id
        self.pos = start
        self.path = [start]
        self.history = [start]
    def step(self):
        if len(self.path) > 1:
            self.path = self.path[1:]
            self.pos = self.path[0]
            self.history.append(self.pos)

# --- Main Execution ---
H, W = 18, 18
world = GridWorld(H, W)
world.add_walls(prob=0.06, seed=31)

agents = [Agent(0, (1, 1)), Agent(1, (1, W-2)), Agent(2, (H-2, 1))]
unexplored = {(r, c) for r in range(H) for c in range(W) if world.grid[r, c] == 0}

# Partition map into columns for each agent
k = len(agents)
cols_per = W // k
regions = []
for i in range(k):
    cols = range(i * cols_per, (i + 1) * cols_per if i < k - 1 else W)
    regions.append({(r, c) for r in range(H) for c in cols if (r, c) in unexplored})

t = 0
while any(regions) and t < 800:
    t += 1
    for i, a in enumerate(agents):
        reg = regions[i]
        if reg:
            target = min(reg, key=lambda x: abs(x[0] - a.pos[0]) + abs(x[1] - a.pos[1]))
            p = bfs(a.pos, {target}, H, W, world.grid)
            if p: a.path = p

        a.step()

        # Mark explored
        if a.pos in reg:
            reg.discard(a.pos)

print("Exploration finished approx. Remaining region sizes:", [len(r) for r in regions])

# Visualization
canvas = np.ones((H, W, 3)) * 0.95
canvas[world.grid == 1] = np.array([0.1, 0.1, 0.1])
for a in agents:
    for h in a.history: canvas[h] = np.array([0.8, 0.9, 0.9])
    r, c = a.pos
    canvas[r, c] = np.array([0.2 + 0.3 * (a.id % 3), 0.2 + 0.4 * ((a.id + 1) % 3), 0.3 + 0.3 * ((a.id + 2) % 3)])

plt.figure(figsize=(6, 6))
plt.imshow(canvas)
plt.title('Map Exploration Final')
plt.axis('off')
plt.show()